In [1]:
%%time

import pandas
import requests
from PIL import Image
from adlfs import AzureBlobFileSystem
from tqdm import tqdm

from common.captioning.azure_descriptions import AzureCaption
from common.schemas.pyarrow_schema import tagging_schema
from common.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas(desc="Progress")

file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

from common.functions.functions import Functions

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

CPU times: total: 3.03 s
Wall time: 3.21 s


In [2]:
# %%time
#
# curated_df = pandas.read_parquet('data/parquet/primary_caption.parquet', filesystem=file_system, engine='pyarrow')
#
# accepted = curated_df.loc[curated_df["accept"] == True]
#
# dropped = accepted.dropna()
# dropped.reset_index(inplace=True, drop=True)
#
# del accepted
#
# accepted = dropped.copy()
# del dropped
#
# display(accepted)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20472,13oersw,AesPleasingAsianGirls,angizni,Glitter,three women in sequin dresses posing for a pic...,cbd83e4de946b42ce318e93d14d78776,/r/AesPleasingAsianGirls/comments/13oersw/glit...,https://i.redd.it/zvr3dt5occ1b1.jpg,13oersw.jpg,data/image/13oersw.jpg,SexyAsianDiffusion,True,True,True,[]
20473,13oxx2r,AesPleasingAsianGirls,theowuriwufoa,Green outfit,araffe woman sitting on a couch in a green outfit,aa460440886847369e59fa8ac2b91f2b,/r/AesPleasingAsianGirls/comments/13oxx2r/gree...,https://i.redd.it/vvy28ulupg1b1.jpg,13oxx2r.jpg,data/image/13oxx2r.jpg,SexyAsianDiffusion,True,True,True,[]
20474,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,araffe woman in a pink bikini standing in a pool,e1244a45a5018d6d35e487217a489db2,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,True,True,True,[]
20475,13qmsn4,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,there is a woman with a bouquet of flowers in ...,a2189d7026d58082336ddcad3667161e,/r/AesPleasingAsianGirls/comments/13qmsn4/song...,https://i.redd.it/cdj3scvnvt1b1.jpg,13qmsn4.jpg,data/image/13qmsn4.jpg,SexyAsianDiffusion,True,True,True,[]


CPU times: total: 438 ms
Wall time: 1.07 s


In [3]:
# %%time
#
#
# display("=== Obtaining Caption Files List ===")
# current_captions = file_system.ls("data/caption")
# display(f"current caption files: {len(current_captions)}")
#
# all_data = []
# filtered_data = []
# for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
# 	image_id = caption_file.split('/')[-1].split('.')[0]
# 	try:
# 		file_size = file_system.size(caption_file)
# 		if file_size == 0:
# 			display(f'Empty file -- removing {image_id}', clear=True)
# 			file_system.rm(caption_file)
# 			continue
# 		caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
# 		caption_data["id"] = image_id
# 		dense_caption_result = caption_data.get('denseCaptionsResult')
# 		metadata = caption_data.get('metadata')
# 		tags_result = caption_data.get('tagsResult')
# 		smart_crop_result = caption_data.get('smartCropsResult')
# 		basic_caption = caption_data.get('captionResult')
# 		_filtered_data = {
# 			"id": image_id,
# 			"captions": [basic_caption],
# 			"dense_captions": dense_caption_result['values'],
# 			"meta": [metadata],
# 			"tags": tags_result['values'],
# 			"smart_crop": smart_crop_result['values']
# 		}
# 		all_data.append(caption_data)
# 		filtered_data.append(_filtered_data)
# 	except Exception as e:
# 		display(f" Exception {e} for {image_id}", clear=True)
# 		continue

'=== Obtaining Caption Files List ==='

'current caption files: 18789'

Reading caption files: 100%|██████████| 18789/18789 [27:56<00:00, 11.21it/s] 

CPU times: total: 6min 39s
Wall time: 28min 9s


In [4]:
# %%time
#
# all_data_from_captions = pandas.DataFrame(data=all_data)
# filtered_data_from_captions = pandas.DataFrame(data=filtered_data)
#
# display(all_data_from_captions)
# display(filtered_data_from_captions)
#
# all_data_from_captions.to_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
# 								  filesystem=file_system)
#
# filtered_data_from_captions.to_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow',
# 									   filesystem=file_system)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,{'text': 'a lizard on a helmet on a football f...,"{'values': [{'boundingBox': {'x': 2654, 'y': 2...","{'stringIndexType': 'TextElements', 'content':...","{'values': [{'text': 'a close-up of a sign', '...",2023-02-01-preview,"{'width': 7407, 'height': 5068}","{'values': [{'name': 'grass', 'confidence': 0....","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 5113, 'y': 2...",10006iy
1,{'text': 'a black and white photo of a hallway...,{'values': []},"{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a white archways with a ...,2023-02-01-preview,"{'width': 1440, 'height': 1800}","{'values': [{'name': 'black and white', 'confi...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
2,{'text': 'a hand holding a red and white flowe...,"{'values': [{'boundingBox': {'x': 457, 'y': 69...","{'stringIndexType': 'TextElements', 'content':...","{'values': [{'text': 'a close up of a flower',...",2023-02-01-preview,"{'width': 3024, 'height': 4032}","{'values': [{'name': 'flower', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 1420, 'y': 2...",1000hda
3,{'text': 'a boat on the shore with green and p...,"{'values': [{'boundingBox': {'x': 261, 'y': 44...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a boat on the shore with...,2023-02-01-preview,"{'width': 1110, 'height': 743}","{'values': [{'name': 'sky', 'confidence': 0.98...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 958, 'y': 43...",1000qpd
4,{'text': 'a woman painting a picture on a beac...,"{'values': [{'boundingBox': {'x': 1229, 'y': 4...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman painting a pictu...,2023-02-01-preview,"{'width': 4032, 'height': 2268}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 1163, 'y': 3...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
18784,"{'text': 'ducks swimming in a pond', 'confiden...",{'values': []},"{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'ducks swimming in a pond...,2023-02-01-preview,"{'width': 4032, 'height': 3024}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 1706, 'y': 2...",zzzmli
18785,"{'text': 'a river running through a forest', '...",{'values': []},"{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a river running through ...,2023-02-01-preview,"{'width': 2000, 'height': 1500}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},zzzoel
18786,{'text': 'a group of chairs outside a blue hou...,"{'values': [{'boundingBox': {'x': 515, 'y': 20...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a chair with a broken fr...,2023-02-01-preview,"{'width': 1080, 'height': 1080}","{'values': [{'name': 'furniture', 'confidence'...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 81, 'y': 612...",zzztxl
18787,"{'text': 'a rock formation in a desert', 'conf...",{'values': []},"{'stringIndexType': 'TextElements', 'content':...","{'values': [{'text': 'a bush in the desert', '...",2023-02-01-preview,"{'width': 4032, 'height': 1960}","{'values': [{'name': 'cloud', 'confidence': 0....","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 1761, 'y': 1...",zzzwub


,id,captions,dense_captions,meta,tags,smart_crop
0,10006iy,[{'text': 'a lizard on a helmet on a football ...,"[{'text': 'a close-up of a sign', 'confidence'...","[{'width': 7407, 'height': 5068}]","[{'name': 'grass', 'confidence': 0.98760414123...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 115..."
1,1000cng,[{'text': 'a black and white photo of a hallwa...,[{'text': 'a white archways with a brick floor...,"[{'width': 1440, 'height': 1800}]","[{'name': 'black and white', 'confidence': 0.9...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
2,1000hda,[{'text': 'a hand holding a red and white flow...,"[{'text': 'a close up of a flower', 'confidenc...","[{'width': 3024, 'height': 4032}]","[{'name': 'flower', 'confidence': 0.9986068606...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 142..."
3,1000qpd,[{'text': 'a boat on the shore with green and ...,[{'text': 'a boat on the shore with green and ...,"[{'width': 1110, 'height': 743}]","[{'name': 'sky', 'confidence': 0.9853595495223...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 173..."
4,1000xd6,[{'text': 'a woman painting a picture on a bea...,[{'text': 'a woman painting a picture on a bea...,"[{'width': 4032, 'height': 2268}]","[{'name': 'outdoor', 'confidence': 0.997976720...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 702..."
...,...,...,...,...,...,...
18784,zzzmli,"[{'text': 'ducks swimming in a pond', 'confide...","[{'text': 'ducks swimming in a pond', 'confide...","[{'width': 4032, 'height': 3024}]","[{'name': 'outdoor', 'confidence': 0.999208092...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 344..."
18785,zzzoel,"[{'text': 'a river running through a forest', ...","[{'text': 'a river running through a forest', ...","[{'width': 2000, 'height': 1500}]","[{'name': 'outdoor', 'confidence': 0.999588847...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 443..."
18786,zzztxl,[{'text': 'a group of chairs outside a blue ho...,"[{'text': 'a chair with a broken frame', 'conf...","[{'width': 1080, 'height': 1080}]","[{'name': 'furniture', 'confidence': 0.9758070...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 51,..."
18787,zzzwub,"[{'text': 'a rock formation in a desert', 'con...","[{'text': 'a bush in the desert', 'confidence'...","[{'width': 4032, 'height': 1960}]","[{'name': 'cloud', 'confidence': 0.99515521526...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 425..."


CPU times: total: 2.81 s
Wall time: 13 s


In [5]:
all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
											 filesystem=file_system)
display(all_data_from_captions)
filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow',
												  filesystem=file_system)
display(filtered_data_from_captions)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
2,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
3,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
4,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
18784,"{'confidence': 0.5420117974281311, 'text': 'du...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 3024, 'w': 4...",2023-02-01-preview,"{'height': 3024, 'width': 4032}","{'values': [{'confidence': 0.9992080926895142,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 76, 'w': 35,...",zzzmli
18785,"{'confidence': 0.4528125524520874, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1500, 'w': 2...",2023-02-01-preview,"{'height': 1500, 'width': 2000}","{'values': [{'confidence': 0.9995888471603394,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},zzzoel
18786,"{'confidence': 0.3901562988758087, 'text': 'a ...","{'values': [{'boundingBox': {'h': 595, 'w': 31...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 346, 'w': 23...",2023-02-01-preview,"{'height': 1080, 'width': 1080}","{'values': [{'confidence': 0.9758070707321167,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 17, 'w': 9, ...",zzztxl
18787,"{'confidence': 0.39278653264045715, 'text': 'a...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 111, 'w': 10...",2023-02-01-preview,"{'height': 1960, 'width': 4032}","{'values': [{'confidence': 0.9951552152633667,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 47, 'w': 31,...",zzzwub


,id,captions,dense_captions,meta,tags,smart_crop
0,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
2,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
3,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
4,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."
...,...,...,...,...,...,...
18784,zzzmli,"[{'confidence': 0.5420117974281311, 'text': 'd...","[{'boundingBox': {'h': 3024, 'w': 4032, 'x': 0...","[{'height': 3024, 'width': 4032}]","[{'confidence': 0.9992080926895142, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 287..."
18785,zzzoel,"[{'confidence': 0.4528125524520874, 'text': 'a...","[{'boundingBox': {'h': 1500, 'w': 2000, 'x': 0...","[{'height': 1500, 'width': 2000}]","[{'confidence': 0.9995888471603394, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 139..."
18786,zzztxl,"[{'confidence': 0.3901562988758087, 'text': 'a...","[{'boundingBox': {'h': 346, 'w': 236, 'x': 395...","[{'height': 1080, 'width': 1080}]","[{'confidence': 0.9758070707321167, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 974..."
18787,zzzwub,"[{'confidence': 0.39278653264045715, 'text': '...","[{'boundingBox': {'h': 111, 'w': 105, 'x': 242...","[{'height': 1960, 'width': 4032}]","[{'confidence': 0.9951552152633667, 'name': 'c...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 190..."


In [6]:
%%time

tags = pandas.DataFrame({'id': filtered_data_from_captions.id, 'tags': filtered_data_from_captions.tags})
out = []
for i, r in tqdm(tags.iterrows(), total=len(tags)):
	if r['tags'] is None:
		continue
	tag = pandas.json_normalize(r['tags'])
	tag['id'] = r['id']
	d = tag.to_dict(orient='records')
	out.extend(d)
converted_tags = pandas.DataFrame(data=out)
display(converted_tags)
converted_tags.to_parquet("data/parquet/tags.parquet", engine='pyarrow', filesystem=file_system)

100%|██████████| 18789/18789 [00:16<00:00, 1171.82it/s]


,confidence,name,id
0,0.987604,grass,10006iy
1,0.923687,outdoor,10006iy
2,0.915661,artificial turf,10006iy
3,0.912514,stadium,10006iy
4,0.880675,sports equipment,10006iy
...,...,...,...
277531,0.903520,yellow,zzzyty
277532,0.859668,twig,zzzyty
277533,0.815015,leaf,zzzyty
277534,0.783504,sitting,zzzyty


CPU times: total: 16.6 s
Wall time: 17.5 s


In [7]:
%%time

dense = pandas.DataFrame(
	{'id': filtered_data_from_captions.id, 'dense_captions': filtered_data_from_captions.dense_captions})

out = []
for i, r in tqdm(dense.iterrows(), total=len(dense)):
	if r['dense_captions'] is None:
		continue
	dense_caption = pandas.json_normalize(r['dense_captions'])
	dense_caption['id'] = r['id']
	d = dense_caption.to_dict(orient='records')
	out.extend(d)

converted = pandas.DataFrame(data=out)
display(converted)
converted.to_parquet("data/parquet/dense_captions.parquet", engine='pyarrow', filesystem=file_system)

100%|██████████| 18789/18789 [00:21<00:00, 884.40it/s]


,confidence,text,boundingBox.h,boundingBox.w,boundingBox.x,boundingBox.y,id
0,0.481536,a close-up of a sign,311,1196,1209,1542,10006iy
1,0.485781,a football helmet on grass,1853,7329,0,3068,10006iy
2,0.559779,a lizard on a helmet on a football field,5068,7407,0,0,10006iy
3,0.519275,a lizard on a ball,935,1461,2609,2192,10006iy
4,0.660839,a lizard on a football helmet,2190,2613,2380,2685,10006iy
...,...,...,...,...,...,...,...
155425,0.459575,a blurry green object,145,206,1395,145,zzzyty
155426,0.583457,a close up of a bird's face,184,331,609,1096,zzzyty
155427,0.628157,a close-up of a leaf,215,367,1346,2143,zzzyty
155428,0.458817,a close up of a plant,785,1045,1291,1553,zzzyty


CPU times: total: 21.8 s
Wall time: 23.6 s


In [8]:
# single_caption_data = pandas.DataFrame(
# 	{
# 		'id': all_data_from_captions['id'],
# 		'azure_caption': [item['text'] for item in all_data_from_captions['captionResult']],
# 		'tags': [[foo['name'] for foo in item['values']] for item in all_data_from_captions['tagsResult']]
# 	})
#
#
# single_caption_data_indexed = single_caption_data.set_index("id")
#
# accepted_indexed = accepted.set_index("id")
#
# for index, row in accepted_indexed.iterrows():
# 	accepted_indexed.at[index, 'azure_caption'] = ""
# 	accepted_indexed.at[index, 'thumbnail_path'] = ""
# 	accepted_indexed.at[index, 'thumbnail_exists'] = False
# 	accepted_indexed.at[index, 'thumbnail_curated'] = False
# 	accepted_indexed.at[index, 'thumbnail_accept'] = False
#	accepted_indexed.at[index, 'additional_captions'] = ['']
#
# accepted_indexed.update(single_caption_data_indexed)
#
# accepted_final = accepted_indexed.reset_index()
#
# display("== Updated With Basic Captions ==")
# display(accepted_final)

'== Updated With Basic Captions =='

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person, human face, clothing, lady, smile, ch...",a woman taking a selfie,,False,False,False,
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person, human face, skin, eyelash, eyebrow, b...",a woman taking a selfie,,False,False,False,[]
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing, person, human face, smile, shoulder...",a woman smiling at camera,,False,False,False,[]
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person, human face, indoor, clothing, woman, ...",a woman taking a selfie,,False,False,False,[]
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person, human face, smile, clothing, woman, i...",a woman taking a selfie,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20472,13oersw,AesPleasingAsianGirls,angizni,Glitter,three women in sequin dresses posing for a pic...,cbd83e4de946b42ce318e93d14d78776,/r/AesPleasingAsianGirls/comments/13oersw/glit...,https://i.redd.it/zvr3dt5occ1b1.jpg,13oersw.jpg,data/image/13oersw.jpg,...,True,True,True,[],,,False,False,False,[]
20473,13oxx2r,AesPleasingAsianGirls,theowuriwufoa,Green outfit,araffe woman sitting on a couch in a green outfit,aa460440886847369e59fa8ac2b91f2b,/r/AesPleasingAsianGirls/comments/13oxx2r/gree...,https://i.redd.it/vvy28ulupg1b1.jpg,13oxx2r.jpg,data/image/13oxx2r.jpg,...,True,True,True,[],,,False,False,False,[]
20474,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,araffe woman in a pink bikini standing in a pool,e1244a45a5018d6d35e487217a489db2,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,...,True,True,True,[],,,False,False,False,[]
20475,13qmsn4,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,there is a woman with a bouquet of flowers in ...,a2189d7026d58082336ddcad3667161e,/r/AesPleasingAsianGirls/comments/13qmsn4/song...,https://i.redd.it/cdj3scvnvt1b1.jpg,13qmsn4.jpg,data/image/13qmsn4.jpg,...,True,True,True,[],,,False,False,False,[]


In [10]:
# accepted_final.to_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
# del accepted_final

KeyboardInterrupt: 

In [ ]:
%%time

accepted_final = pandas.read_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system,
									 schema=tagging_schema)
display(accepted_final)

In [ ]:
%%time

def get_aspect_ratio(x: object):
	return x['crops'][0]['aspectRatio']


def get_bounding_box(x: object):
	return x['crops'][0]['boundingBox']

In [ ]:
cropping = pandas.DataFrame(
	{'id': filtered_data_from_captions['id'], 'crops': filtered_data_from_captions['smart_crop']}).set_index('id',
																											 drop=False)

cropping['aspectRatio'] = cropping.progress_apply(lambda x: get_aspect_ratio(x), axis=1)
cropping['bounding_box'] = cropping.progress_apply(lambda x: get_bounding_box(x), axis=1)
cropping['x'] = cropping.progress_apply(lambda x: x['bounding_box']['x'], axis=1)
cropping['y'] = cropping.progress_apply(lambda x: x['bounding_box']['y'], axis=1)
cropping['w'] = cropping.progress_apply(lambda x: x['bounding_box']['w'], axis=1)
cropping['h'] = cropping.progress_apply(lambda x: x['bounding_box']['h'], axis=1)

display(cropping)

In [ ]:
def create_thumbnail(target_image_id: str, file_names: list, crops: pandas.DataFrame, curated_data: pandas.DataFrame):
	_file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

	out_path = f"data/image/thumbnail/{target_image_id}.jpg"
	try:
		if target_image_id is None or out_path in file_names:
			# print(f'Image {target_image_id} already exists, skipping')
			return out_path

		cropping_information = cropping.loc[crops['id'] == target_image_id]
		if cropping_information is None or len(cropping_information) == 0:
			# print(f'No cropping information for {target_image_id}, skipping')
			return "/data/nope"

		record = curated_data.loc[curated_data['id'] == target_image_id]
		record_path = record.to_dict(orient='records')[0]['path']
		image_url = file_system.url(record_path)
		original_image = Image.open(requests.get(image_url, stream=True).raw)
		copied_image = original_image.copy()
		original_image.close()

		cropped = copied_image.crop((cropping_information['x'].values[0],
									 cropping_information['y'].values[0],
									 cropping_information['x'].values[0] +
									 cropping_information['w'].values[0],
									 cropping_information['y'].values[0] +
									 cropping_information['h'].values[0]))
		copied_image.close()

		resized = cropped.resize((512, 512), 1)
		resized.save('temp.jpg')
		resized.close()
		file_system.upload('temp.jpg', out_path, overwrite=True)
		print(f'Thumbnail created for {target_image_id}')
		return out_path

	except Exception as ex:
		print(f'Error creating thumbnail for {target_image_id}: {ex}')
		return "/data/nope"

In [ ]:
file_names = file_system.ls('data/image/thumbnail')

accepted_final['thumbnail_path'] = accepted_final.progress_apply(
	lambda x: create_thumbnail(x['id'], file_names, cropping, accepted_final), axis=1)

display(accepted_final)

In [ ]:
%%time

accepted_final['thumbnail_exists'] = accepted_final.progress_apply(lambda x: file_system.exists(x['thumbnail_path']),
																   axis=1)

display(accepted_final)

In [ ]:
%%time

accepted_copy = accepted_final.copy()

accepted_copy['additional_captions'] = accepted_copy.progress_apply(lambda x: [], axis=1)

accepted_slice = accepted_copy.loc[accepted_copy['thumbnail_exists'] == True, tagging_schema.names]

accepted_slice.dropna(inplace=True)

accepted_slice.reset_index(inplace=True, drop=True)

display(accepted_slice)

In [ ]:
%%time

# TODO: Only update the this accepted slice if there are new captions, we don't want to overwrite the curated data

In [ ]:
%%time

file_system.cp('data/parquet/thumbnail_curation.parquet', 'data/parquet/thumbnail_curation.parquet.bak')

# accepted_slice.to_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)

In [ ]:
display(accepted_slice)

In [ ]:
# @title Initalize Helpers

%%time

class Functions:
	def get_hash_from_path(self, in_path: str):
		if os.path.exists(in_path):
			with open(in_path, 'rb') as f_:
				content = f_.read()
				result = hashlib.md5(content).hexdigest()
				return result, content
		else:
			return ""

	def fetch_image(self, x: object, file_list__, file_system) -> object:

	try:
		url = x['original_url']
		subreddit = x['subreddit']
		image_id = x['id']
		os.makedirs(f"temp\\image\\{subreddit}", exist_ok=True)
		temp_path = f"temp\\image\\{subreddit}\\{image_id}.jpg"
		out_path = f"data/image/{image_id}.jpg"
		if os.path.exists(temp_path):
			md5, content = self.get_hash_from_path(temp_path)
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				if out_path in file_list__:
					return out_path
				else:
					file_system.upload(temp_path, out_path, overwrite=True)
					return out_path
			else:
				return ""
		else:
			response = requests.get(url)
			md5 = hashlib.md5(response.content).hexdigest()
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				try:
					raw_image = Image.open(BytesIO(response.content))
					raw_image.save(temp_path)
					raw_image.close()
					if out_path in file_list__:
						return out_path
					else:
						file_system.upload(temp_path, out_path)
						return out_path
				except Exception as ex:
					message = self.write_log_message(x['id'], x['subreddit'], "Failure in fetch_image", ex)
					display(message, clear=True)
					return ""
			else:
				return ""
	except Exception as ex:
		message = self.write_log_message(x['id'], x['subreddit'], "Failure in fetch_image", ex)
		display(message, clear=True)
		return ""

	def get_name_for_image(self, x: object, file_list__) -> str:
		path = x['path']
		if path != "" and path in file_list__:
			return os.path.basename(path)
		else:
			return ""

	def set_exists(self, x: object) -> bool:
		try:
			sub_reddit = x['subreddit']
			record_id = x['id']
			temp_path = f"temp\\image\\{sub_reddit}\\{record_id}.jpg"
			return os.path.exists(temp_path)
		except Exception as ex:
			return False

	def set_hash(self, x: object):
		sub_reddit = x['subreddit']
		record_id = x['id']
		temp_path = f"temp\\image\\{sub_reddit}\\{record_id}.jpg"
		if os.path.exists(temp_path):
			return hashlib.md5(open(temp_path, 'rb').read()).hexdigest()
		else:
			return ""

	def add_source(self, x: object, source_list) -> str:
		sub_reddit = x['subreddit']
		for source in source_list:
			data_source = source['data']
			source_name = source['name']
			if sub_reddit in data_source:
				return source_name
		return ""

	def write_log_message(self, submission_id: str, subreddit: str, message: str, exception: Exception) -> str:
		date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
		return f"{date_time}\t{subreddit}\t{submission_id}\t{message}\t{exception}\n"

	def apply_caption(self, x: object, caption_routine: list[BlipCaption]) -> str:
		exists = x['exists']
		if not exists:
			return ""
		sub_reddit = x['subreddit']
		record_id = x['id']
		os.makedirs(f"temp\\image\\{sub_reddit}\\", exist_ok=True)
		temp_path = f"temp\\image\\{sub_reddit}\\{record_id}.jpg"

		if os.path.exists(temp_path):
			try:
				result = random.choice(caption_routine).caption_image(temp_path)
				return result
			except Exception as ex:
				message = self.write_log_message(x['id'], x['subreddit'], "Failure in apply_caption", ex)
				display(message, clear=True)
				return ""
		else:
			message = self.write_log_message(x['id'], x['subreddit'], f"Image not found in {temp_path}",
											 Exception("No image"))
			display(message, clear=True)
			return ""

	def fix_path(self, x: object, fl: []) -> str:
		current_path = x['path']
		exists = x['exists']
		if current_path in fl:
			return current_path
		else:
			image_id = x['id']
			if exists:
				return f"data/image/{image_id}.jpg"
			else:
				return ""
		current_path = x['path']
		exists = x['exists']
		if current_path in fl:
			return current_path
		else:
			image_id = x['id']
			if exists:
				return f"data/image/{image_id}.jpg"
			else:
				return ""


class AzureFileStorageAdapter(object):
	def __init__(self, container_name: str = "data"):
		self.__account_name: str = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
		self.__account_key: str = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]
		self.container_name: str = container_name

	def get_file_storage(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(
			account_name=self.__account_name,
			account_key=self.__account_key,
			container_name=self.container_name)

	def get_file_storage_root(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(account_name=self.__account_name, account_key=self.__account_key,
								   container_name=self.container_name)


schema = pyarrow.schema(
	[
		pyarrow.field("id", pyarrow.string()),
		pyarrow.field("subreddit", pyarrow.string()),
		pyarrow.field("author", pyarrow.string()),
		pyarrow.field("title", pyarrow.string()),
		pyarrow.field("caption", pyarrow.string()),
		pyarrow.field("hash", pyarrow.string()),
		pyarrow.field("permalink", pyarrow.string()),
		pyarrow.field("original_url", pyarrow.string()),
		pyarrow.field("image_name", pyarrow.string()),
		pyarrow.field("path", pyarrow.string()),
		pyarrow.field("model", pyarrow.string()),
		pyarrow.field("exists", pyarrow.bool_()),
		pyarrow.field("curated", pyarrow.bool_()),
		pyarrow.field("accept", pyarrow.bool_()),
		pyarrow.field("tags", pyarrow.list_(pyarrow.string()))
	]
)

In [ ]:
import os
	def fetch_image(self, x: object, file_list__, file_system__) -> object:
		try:
			url = x['original_url']
			subreddit = x['subreddit']
			image_id = x['id']
			os.makedirs(f"temp/image/{subreddit}", exist_ok=True)
			temp_path = f"temp/image/{subreddit}/{image_id}.jpg"
			out_path = f"data/image/{image_id}.jpg"
			if os.path.exists(temp_path):
				md5, content = self.get_hash_from_path(temp_path)
				if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
					if out_path in file_list__:
						return out_path
					else:
						file_system.upload(temp_path, out_path, overwrite=True)
						return out_path
				else:
					return ""
			else:
				response = requests.get(url)
				md5 = hashlib.md5(response.content).hexdigest()
				if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
					try:
						raw_image = Image.open(BytesIO(response.content))
						raw_image.save(temp_path)
						raw_image.close()
						if out_path in file_list__:
							return out_path
						else:
							file_system__.upload(temp_path, out_path)
							return out_path
					except Exception as ex:
						message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
						display(message, clear=True)
						return ""
				else:
					return ""
		except Exception as ex:
			message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
			display(message, clear=True)
			return ""